In [2]:
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import matplotlib.image as mpimg
from lane_lines import Lane_Lines
%matplotlib inline

SyntaxError: unexpected EOF while parsing (lane_lines.py, line 185)

Import Pickle

In [ ]:
dist_pickle = pickle.load( open( "camera_cal/New_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]
presp_M = dist_pickle["presp_M"]
persp_M_inv = dist_pickle["persp_M_inv"]

In [ ]:
def s_select(img, thresh=(100, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    s_channel = hls[:,:,2]
    # 3) Return a binary image of threshold result
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
#     binary_output = np.copy(img) # placeholder line
    return binary_output

def l_select(img, thresh=(120, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    l_channel = hls[:,:,1]
    # 3) Return a binary image of threshold result
    binary_output = np.zeros_like(l_channel)
    binary_output[(l_channel > thresh[0]) & (l_channel <= thresh[1])] = 1
#     binary_output = np.copy(img) # placeholder line
    return binary_output

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img,cv2.CV_64F,1,0))
    else:
        abs_sobel = np.absolute(cv2.Sobel(img,cv2.CV_64F,0,1))
        
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    sobelx = cv2.Sobel(img,cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(img,cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag>=mag_thresh[0]) & (gradmag<=mag_thresh[1])] = 1
    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    sobelx = cv2.Sobel(img,cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(img,cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    absgraddir = np.arctan2(abs_sobely, abs_sobelx)
    dir_binary = np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return dir_binary
    

In [ ]:
def comb_threshold(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    height, width = gray.shape
    s_channel = hls[:,:,2]
    l_channel = hls[:,:,1]
    
    # img to s threshold
    s_binary = s_select(img,thresh=(150, 255))
    
    # img to l threshold
    l_binary = l_select(img,thresh=(120, 255))   
    
    # schannel to x threshold
    ksize = 9  # Choose a larger odd number to smooth gradient measurements
    gradx = abs_sobel_thresh(s_channel, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    
    # combined
    combined = np.zeros_like(s_binary)
    combined[((gradx == 1) | (s_binary == 1)) & ((l_binary == 1) & (s_binary == 1))] = 1

    # apply the region of interest mask
    mask = np.zeros_like(combined)
    region_of_interest_vertices = np.array([[0,height-1], [width/2, int(0.5*height)], [width-1, height-1]], dtype=np.int32)
    cv2.fillPoly(mask, [region_of_interest_vertices], 1)
    thresholded = cv2.bitwise_and(combined, mask)
    
    return thresholded

In [ ]:
def persp_view(img):
    img_size = (img.shape[1], img.shape[0])
    # Vertices extracted manually for performing a perspective transform
    bottom_left = [200,720]
    bottom_right = [1110, 720]
    top_left = [565, 470]
    top_right = [722, 470]

    source = np.float32([bottom_left,bottom_right,top_right,top_left])

    pts = np.array([bottom_left,bottom_right,top_right,top_left], np.int32)
    pts = pts.reshape((-1,1,2))
    img_pts = img.copy()
    cv2.polylines(img_pts,[pts],True,(255,0,0), thickness=3)

    # Destination points are chosen such that straight lanes appear more or less parallel in the transformed image.
    bottom_left = [320,720]
    bottom_right = [920, 720]
    top_left = [320, 1]
    top_right = [920, 1]

    dst = np.float32([bottom_left,bottom_right,top_right,top_left])
    M = cv2.getPerspectiveTransform(source, dst)
    M_inv = cv2.getPerspectiveTransform(dst, source)
#     img_size = (image_shape[1], image_shape[0])
    
    warped = cv2.warpPerspective(img, M, img_size , flags=cv2.INTER_LINEAR)
    
    return warped,img_pts,M, M_inv

In [ ]:
def pipeline(img):

    
    global past_left_fit
    global past_right_fit
    thresholded = comb_threshold(img)
    undist = cv2.undistort(thresholded, mtx, dist, None, mtx)
    binary_w,img_pts,presp_M,persp_M_inv = persp_view(undist)
    
    binary_warped = Lane_Lines(np.copy(binary_w),past_left_fit,past_right_fit)
    out_img,plf,prf = binary_warped.find_lane_pixels()
#     past_left_fit.clear()
    past_left_fit = plf
#     past_right_fit.clear()
    past_right_fit = prf
    
    left_curverad, right_curverad, center_offset_m = binary_warped.measure_curvature_real()
    radius_string = 'Radius of road = %.2f m'% ((left_curverad+right_curverad)/2)
    offset_string = 'Center Offset = %.2f m'% center_offset_m
    
    dstack_thresh = np.dstack((binary_w,binary_w,binary_w))*255
    dstack_thresh_size = (dstack_thresh.shape[1], dstack_thresh.shape[0])
    left_line_window = np.array(np.transpose(np.vstack([binary_warped.lx, binary_warped.ploty])))
    right_line_window = np.array(np.flipud(np.transpose(np.vstack([binary_warped.rx, binary_warped.ploty]))))
    line_points = np.vstack((left_line_window, right_line_window))
    cv2.fillPoly(dstack_thresh, np.int_([line_points]), [0,255, 0])
    un_warped = cv2.warpPerspective(dstack_thresh, persp_M_inv, dstack_thresh_size , flags=cv2.INTER_LINEAR)
    
    result = cv2.addWeighted(img, 0.1, un_warped, 1, 0)
    cv2.putText(result,radius_string, (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=2)
    cv2.putText(result, offset_string, (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=2)
    
    return result

In [ ]:
img = mpimg.imread('test_images/test1.jpg')
dist_pickle = pickle.load( open( "camera_cal/New_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]
presp_M = dist_pickle["presp_M"]
persp_M_inv = dist_pickle["persp_M_inv"]
    
past_left_fit = []
past_right_fit = []
thresholded = comb_threshold(img)
undist = cv2.undistort(thresholded, mtx, dist, None, mtx)
# binary_w,img_pts,presp_M,persp_M_inv = persp_view(undist)
undist_size = (undist.shape[1], undist.shape[0])
binary_w = cv2.warpPerspective(undist, presp_M, undist_size, flags = cv2.INTER_LINEAR)

plt.figure(1)
plt.imshow(img)
plt.title('IMG')
plt.figure(2)
plt.imshow(thresholded)
plt.title('THRESHOLDED')
plt.figure(3)
plt.imshow(undist)
plt.title('THRESHOLDED AND UNDISTORTED')
plt.figure(4)
plt.imshow(binary_w)
plt.title('BINARY_W AND PERSPECTIVE')


# binary_warped = Lane_Lines(np.copy(binary_w[:,:,1]))
binary_warped = Lane_Lines(np.copy(binary_w),past_left_fit,past_right_fit)
out_img,plf,prf = binary_warped.find_lane_pixels()
left_curverad, right_curverad, center_offset_m = binary_warped.measure_curvature_real()
radius_string = 'Radius of road = %.2f m'% ((left_curverad+right_curverad)/2)
offset_string = 'Center Offset = %.2f m'% center_offset_m

print('Left Lane =' ,"%.2f"% left_curverad, 'm and Right Lane =', "%.2f"% right_curverad, 'm')
print(radius_string)
print(offset_string)
plt.figure(5)
plt.imshow(out_img)
plt.title('LANE LINES')

dstack_thresh = np.dstack((binary_w,binary_w,binary_w))*255
dstack_thresh_size = (dstack_thresh.shape[1], dstack_thresh.shape[0])
left_line_window = np.array(np.transpose(np.vstack([binary_warped.lx, binary_warped.ploty])))
right_line_window = np.array(np.flipud(np.transpose(np.vstack([binary_warped.rx, binary_warped.ploty]))))
line_points = np.vstack((left_line_window, right_line_window))
cv2.fillPoly(dstack_thresh, np.int_([line_points]), [0,255, 0])
un_warped = cv2.warpPerspective(dstack_thresh, persp_M_inv, dstack_thresh_size , flags=cv2.INTER_LINEAR)

plt.figure(6)
plt.imshow(un_warped)
plt.title('UNWARPED LANE')

result = cv2.addWeighted(img, 1, un_warped, 0.3, 0)
cv2.putText(result,radius_string, (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=2)
cv2.putText(result, offset_string, (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=2)

plt.figure(7)
plt.imshow(result)
plt.title('UNWARPED IMAGE WITH LANE')

In [ ]:
#for image
img = mpimg.imread('test_images/test1.jpg')
past_left_fit = []
past_right_fit = []
result = pipeline(img)
plt.figure(10)
plt.imshow(result)
plt.title('Final OP')

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
output = 'project_video_output.mp4'
# clip1 = VideoFileClip("project_video.mp4")
clip1 = VideoFileClip("project_video.mp4").subclip(20,30)
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))